In [82]:
import os
import cv2
import pandas as pd
import numpy as np
from matplotlib import pyplot
from ipywidgets import interact
from PIL import Image

In [ ]:
os.getcwd()

In [83]:
root = '/opt/ml/input'
dataset_path = "data"
file = pd.read_csv('../../submission/submission_SeMask-FAPN_trash_dataV2_WarmupPolyLR_1e-5_fold-1-model_best_9099iter_fold-0.csv')

In [ ]:
for i in range(10):
    row = file.iloc[i]
    # print(row)


In [85]:
PALLETE = [[  0,   0,   0], [128,   0,   0], [  0, 128,   0], [128, 128,   0],
           [  0,   0, 128], [128,   0, 128], [  0, 128, 128], [128, 128, 128],
           [ 64,   0,   0], [192,   0,   0],[ 64, 128,   0]]

def show_img(i):
    row = file.iloc[i]
    img_id = row['image_id']
    
    pred = row['PredictionString']
    pred = np.array(pred.split(' '))
    pred = np.array(list(map(lambda x: PALLETE[int(x)], pred)))
    # pred = pred.reshape((512,512,3))
    pred = pred.reshape((256, 256, 3))
    print(img_id)
    pyplot.imshow(pred)

interact(show_img, i=range(100))

interactive(children=(Dropdown(description='i', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,…

<function __main__.show_img(i)>

In [ ]:
pred = file["PredictionString"][0]
anns = np.array(pred.split(' '))
anns = np.array(list(map(lambda x: PALLETE[int(x)], anns)), dtype=np.uint8)
anns = anns.reshape((512, 512, 3))
print(anns.dtype)
image = Image.fromarray(anns)
image2 = image.convert("RGB").convert("P", palette=Image.ADAPTIVE)
print(image2.size)
image2.save(f'pseudo_0000_2.png')

In [ ]:
for index, (image, pred) in enumerate(zip(file["image_id"], file["PredictionString"])):
    anns = np.array(pred.split(' '))
    anns = np.array(list(map(lambda x: PALLETE[int(x)], anns)), dtype=np.uint8)
    anns = anns.reshape((512, 512, 3))
    if index == 10:
        print(anns.dtype)
    if not os.path.exists(os.path.join(root, dataset_path, 'pseudo_annotations_4')):
        os.mkdir(os.path.join(root, dataset_path, 'pseudo_annotations_4'))
    image = Image.fromarray(anns.astype(np.uint8))
    image2 = image.convert("RGB").convert("P", palette=Image.ADAPTIVE)
    image2.save(os.path.join(root, dataset_path, 'pseudo_annotations_4', f'pseudo_{index:04}.png'), 'png')


In [ ]:
import PIL
print(PIL.__version__)

In [ ]:
for index, image in enumerate(file["image_id"]):
    temp_image = cv2.imread(os.path.join(root, dataset_path, image))
    if not os.path.exists(os.path.join(root, dataset_path, 'pseudo_images')):
        os.mkdir(os.path.join(root, dataset_path, 'pseudo_images'))
    temp_image = temp_image[:,:,::-1]
    cv2.imwrite(os.path.join(root, dataset_path, 'pseudo_images', f'pseudo_{index:04}.jpg'), temp_image)


In [ ]:
import shutil
if not os.path.exists(os.path.join(root, dataset_path, 'pseudo_images')):
    os.mkdir(os.path.join(root, dataset_path, 'pseudo_images'))
        
for index, image in enumerate(file["image_id"]):
    img_origin_path = os.path.join(root, dataset_path, image)
    img_output_path = os.path.join(root, dataset_path, 'pseudo_images', f'pseudo_{index:04}.jpg')
    shutil.copy(img_origin_path, img_output_path)


In [ ]:
import torch

img_np = cv2.imread(os.path.join(root, dataset_path, "pseudo_annotations", "pseudo_0000.png"))
img_tensor = torch.from_numpy(img_np)
img_tensor2 = torch.Tensor(img_np)
print(img_tensor.size())
print(img_tensor2.size())

In [ ]:
image_origin = cv2.imread(os.path.join(root, dataset_path, "pseudo_images", "pseudo_0000.jpg"))
img_tensor = torch.from_numpy(img_np)
img_tensor2 = torch.Tensor(img_np)
print(img_tensor.size())
print(img_tensor2.size())

In [ ]:
image_rgb = Image.open(os.path.join(root, dataset_path, "pseudo_annotations_3", "pseudo_0000.png"))
image_rgb_array = np.array(image_rgb)

image_rgb.show()
image_to_tensor = torch.Tensor(image_rgb_array)
print(image_to_tensor.shape)